In [4]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-01 04:16:17--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  72.8MB/s    in 87s     

2022-05-01 04:17:45 (73.5 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-01 04:18:16--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [5]:
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt

--2022-05-01 04:18:29--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt.1’

map_clsloc.txt.1    100%[===================>]  23.79K  --.-KB/s    in 0.001s  

2022-05-01 04:18:29 (20.7 MB/s) - ‘map_clsloc.txt.1’ saved [24366/24366]



In [2]:
from torchvision import models
import torch
from PIL import Image

In [10]:
target_model = models.efficientnet_b1(pretrained=True)
target_model.eval()

Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-533bc792.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-533bc792.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [6]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [7]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')
validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
classes_mapping_list = []
classes_mapping = {}
for i in range(len(mapping_list_from)):
  classes_mapping[mapping_list_from[i].split(" ")[2].replace('\n',"")] = mapping_list_from[i].split(" ")[1]
for i in range(len(classes)):
  classes_mapping_list.append(classes_mapping[classes[i].split(", ")[1]])

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

In [12]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  # print(img_path)
  # img = Image.open("dog.jpg")
  img = Image.open(img_path).convert('RGB')
  img_t = transform(img)
  batch_t = torch.unsqueeze(img_t, 0)

  out = target_model(batch_t)

  _, index = torch.topk(input=out,k=5)

  # print(index[0].tolist())

  if classes_mapping_list[index[0][0]] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      if classes_mapping_list[index[0][ii]] == validation_list[i]:
        top_5 = top_5 + 1
        break
  # print(correct, top_5)
  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

1%, Top 1:0.788 Top 5:0.942
2%, Top 1:0.778 Top 5:0.936
3%, Top 1:0.786 Top 5:0.9393333333333334
4%, Top 1:0.775 Top 5:0.936
5%, Top 1:0.7772 Top 5:0.936
6%, Top 1:0.776 Top 5:0.9363333333333334
7%, Top 1:0.7757142857142857 Top 5:0.9362857142857143
8%, Top 1:0.77575 Top 5:0.9355
9%, Top 1:0.7764444444444445 Top 5:0.9362222222222222
10%, Top 1:0.7762 Top 5:0.9334
11%, Top 1:0.7763636363636364 Top 5:0.9338181818181818
12%, Top 1:0.7766666666666666 Top 5:0.9355
13%, Top 1:0.7784615384615384 Top 5:0.9343076923076923
14%, Top 1:0.7788571428571428 Top 5:0.9337142857142857
15%, Top 1:0.7788 Top 5:0.9338666666666666
16%, Top 1:0.779375 Top 5:0.93375
17%, Top 1:0.78 Top 5:0.9338823529411765
18%, Top 1:0.7788888888888889 Top 5:0.9341111111111111
19%, Top 1:0.7797894736842105 Top 5:0.9334736842105263
20%, Top 1:0.779 Top 5:0.9325
21%, Top 1:0.7786666666666666 Top 5:0.9321904761904762
22%, Top 1:0.779 Top 5:0.9321818181818182
23%, Top 1:0.7782608695652173 Top 5:0.9323478260869565
24%, Top 1:0.7778

In [13]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:77.546%
Top 5 Accuracy:93.404%
